In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import json

In [2]:
starbucks = pd.read_csv('./startbucks_new2.csv')

In [3]:
us_df = starbucks.iloc[7534:11238]

In [4]:
fr_df = starbucks[starbucks['countryCode'] == 'FR']

In [6]:
def str_to_json(json_str):
    if pd.isna(json_str):
        return 'Null'
    else:
        str_for_json = json_str.replace("'", '"').replace("True", '"True"').replace("False", '"False"').replace("None", '"None"')[1:-1]
        dd = pd.read_json(str_for_json, lines=True)
        hours_dd = dd['hours']
        return hours_dd

In [7]:
hours_fr = fr_df.apply(lambda x: str_to_json(x['schedule']), axis=1) 
result_fr = pd.merge(fr_df, hours_fr, left_index=True, right_index=True)

In [8]:
hours_us = us_df.apply(lambda x: str_to_json(x['schedule']), axis=1) 
result_us = pd.merge(us_df, hours_us, left_index=True, right_index=True)

In [9]:
# concat two dataframe
melody_result = pd.concat([result_us,result_fr])

In [10]:
def new_column(result):
    result.columns = ['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule','slug', 
                  'latitude', 'longitude', 'streetAddressLine1','streetAddressLine2', 'streetAddressLine3', 
                  'city','countrySubdivisionCode', 'postalCode', 'currentTimeOffset','windowsTimeZoneId', 
                  'olsonTimeZoneId', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon']
    return result

In [11]:
new_column(melody_result)

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,...,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId,Tue,Wed,Thu,Fri,Sat,Sun,Mon
7534,7535,74306-78900,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 8:0...",safeway-fort-lupton-1045-1300-dexter-st-fort-l...,40.079099,-104.802889,1300 Dexter St,NaN,...,-360,Mountain Standard Time,GMT-07:00 America/Denver,6:00 AM to 8:00 PM,6:00 AM to 8:00 PM,6:00 AM to 8:00 PM,6:00 AM to 8:00 PM,6:00 AM to 8:00 PM,6:00 AM to 8:00 PM,6:00 AM to 8:00 PM
7535,7536,14436-114804,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 9:0...",bay-road-tittabawassee-2685-tittabawassee-sagi...,43.479563,-83.968724,2685 Tittabawassee,NaN,...,-240,Eastern Standard Time,GMT-05:00 America/Detroit,5:30 AM to 9:00 PM,5:30 AM to 9:00 PM,5:30 AM to 9:00 PM,5:30 AM to 10:00 PM,5:30 AM to 10:00 PM,5:30 AM to 9:00 PM,5:30 AM to 9:00 PM
7536,7537,55799-183824,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 9:0...",target-saginaw-t-347-2772-tittabawassee-rd-sag...,43.483322,-83.970879,2772 Tittabawassee Rd,NaN,...,-240,Eastern Standard Time,GMT-05:00 America/Detroit,8:00 AM to 9:00 PM,8:00 AM to 9:00 PM,8:00 AM to 9:00 PM,8:00 AM to 9:00 PM,8:00 AM to 9:00 PM,8:00 AM to 8:00 PM,8:00 AM to 9:00 PM
7537,7538,78052-108426,US,LS,"[{'dayName': 'Today', 'hours': '7:30 AM to 7:3...",saginaw-valley-state-university-7400-bay-rd-sa...,43.513020,-83.959972,7400 Bay Rd,NaN,...,-240,Eastern Standard Time,GMT-05:00 America/Detroit,7:30 AM to 7:30 PM,7:30 AM to 7:30 PM,7:30 AM to 7:30 PM,7:30 AM to 7:30 PM,Closed,12:00 PM to 6:00 PM,7:30 AM to 7:30 PM
7538,7539,51497-277347,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 9:0...",state-center-4961-state-street-saginaw-mi-4860...,43.436290,-84.013049,4961 State Street,Suite 100,...,-240,Eastern Standard Time,GMT-05:00 America/Detroit,5:30 AM to 9:00 PM,5:30 AM to 9:00 PM,5:30 AM to 9:00 PM,5:30 AM to 9:00 PM,6:00 AM to 9:00 PM,6:00 AM to 9:00 PM,5:30 AM to 9:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17732,17733,50655-275439,FR,LS,"[{'dayName': 'Today', 'hours': '10:00 AM to 7:...",miramas-mac-arthur-glen-outlet-sc-zi-les-molie...,43.594709,4.994581,"ZI les Molieres, 6 Boulevard d'Espagne",NaN,...,60,Romance Standard Time,GMT+00:00 Europe/Paris,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,10:00 AM to 8:00 PM,Closed,10:00 AM to 7:00 PM
17733,17734,60412-294687,FR,LS,"[{'dayName': 'Today', 'hours': '9:00 AM to 8:0...",avenue-83-300-avenue-de-luniversite-la-valette...,43.137051,6.004876,300 avenue de l'universite,NaN,...,60,Romance Standard Time,GMT+00:00 Europe/Paris,9:00 AM to 8:00 PM,9:00 AM to 8:00 PM,9:00 AM to 8:00 PM,9:00 AM to 8:00 PM,9:00 AM to 8:00 PM,11:00 AM to 7:00 PM,9:00 AM to 8:00 PM
17734,17735,54711-288383,FR,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 8:0...",morieres-les-avignon-a-7-a-7-aire-de-morieres-...,43.922646,4.912338,A7 Aire de Morieres,NaN,...,60,Romance Standard Time,GMT+00:00 Europe/Paris,7:00 AM to 8:00 PM,7:00 AM to 8:00 PM,7:00 AM to 8:00 PM,7:00 AM to 8:00 PM,7:00 AM to 8:00 PM,7:00 AM to 8:00 PM,7:00 AM to 8:00 PM
17735,17736,19522-186944,FR,LS,"[{'dayName': 'Today', 'hours': '10:00 AM to 7:...",saint-germain-en-laye-19-rue-de-la-salle-saint...,48.897211,2.094023,19 rue de la Salle,NaN,...,60,Romance Standard Time,GMT+00:00 Europe/Paris,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,10:00 AM to 7:00 PM,9:00 AM to 7:00 PM,10:00 AM to 7:00 PM


In [12]:
day_list=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [13]:
# separate day to day_open, day_close
for i in day_list:
    time = melody_result[i].str.split(' to ', expand=True)
    open_name = i +'_open'
    close_name = i + '_close'
    melody_result[open_name]=time[0]
    melody_result[close_name]=time[1]
    melody_result=melody_result.drop([i],axis=1)

In [14]:
# convert AM, PM to 24hr clock
for j in day_list:
    open_name = j + '_open'
    close_name = j + '_close'
    melody_result.loc[melody_result[open_name]=='Closed', close_name] = 'Closed'
    melody_result.loc[melody_result[open_name].str.contains('M', na=False), open_name] = pd.to_datetime(melody_result[open_name], errors='coerce').dt.strftime('%H:%M')
    melody_result.loc[melody_result[close_name].str.contains('M', na=False), close_name] = pd.to_datetime(melody_result[close_name], errors='coerce').dt.strftime('%H:%M')

In [15]:
# other cleaning processes
def clean_process(df):
    gmt = df['olsonTimeZoneId'].str.split(' ', expand=True)
    df['GMT'] = gmt[0]
    df['GMT_region'] = gmt[1]   
    df['city'] = df['city'].str.lower()
    cols = ['streetAddressLine1', 'streetAddressLine2', 'streetAddressLine3']
    df["address"] = df[cols].apply(lambda x: '|'.join(x.dropna()), axis=1)
    df.drop(['slug', 'currentTimeOffset', 'olsonTimeZoneId', 'streetAddressLine1','streetAddressLine2','streetAddressLine3'], axis=1, inplace=True)

In [16]:
clean_process(melody_result)

In [17]:
# reorder columns
melody_result = melody_result[['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude',
       'longitude', 'address', 'city', 'countrySubdivisionCode', 'postalCode',
       'windowsTimeZoneId', 'GMT', 'GMT_region', 'Mon_open', 'Mon_close',
       'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close', 'Thu_open',
       'Thu_close', 'Fri_open', 'Fri_close', 'Sat_open', 'Sat_close',
       'Sun_open', 'Sun_close']]

In [18]:
melody_result = melody_result.where(pd.notnull(melody_result), 'Null')
melody_result.isnull().sum()

No.                       0
storeNumber               0
countryCode               0
ownershipTypeCode         0
latitude                  0
longitude                 0
address                   0
city                      0
countrySubdivisionCode    0
postalCode                0
windowsTimeZoneId         0
GMT                       0
GMT_region                0
Mon_open                  0
Mon_close                 0
Tue_open                  0
Tue_close                 0
Wed_open                  0
Wed_close                 0
Thu_open                  0
Thu_close                 0
Fri_open                  0
Fri_close                 0
Sat_open                  0
Sat_close                 0
Sun_open                  0
Sun_close                 0
dtype: int64

In [19]:
melody_result

,No.,storeNumber,countryCode,ownershipTypeCode,latitude,longitude,address,city,countrySubdivisionCode,postalCode,...,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
7534,7535,74306-78900,US,LS,40.079099,-104.802889,1300 Dexter St,fort lupton,CO,806211501,...,06:00,20:00,06:00,20:00,06:00,20:00,06:00,20:00,06:00,20:00
7535,7536,14436-114804,US,CO,43.479563,-83.968724,2685 Tittabawassee,saginaw,MI,486048217,...,05:30,21:00,05:30,21:00,05:30,22:00,05:30,22:00,05:30,21:00
7536,7537,55799-183824,US,LS,43.483322,-83.970879,2772 Tittabawassee Rd,saginaw,MI,48604,...,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,20:00
7537,7538,78052-108426,US,LS,43.513020,-83.959972,7400 Bay Rd,saginaw,MI,486049733,...,07:30,19:30,07:30,19:30,07:30,19:30,Closed,Closed,12:00,18:00
7538,7539,51497-277347,US,CO,43.436290,-84.013049,4961 State Street|Suite 100,saginaw,MI,486033892,...,05:30,21:00,05:30,21:00,05:30,21:00,06:00,21:00,06:00,21:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17732,17733,50655-275439,FR,LS,43.594709,4.994581,"ZI les Molieres, 6 Boulevard d'Espagne",miramas,U,13140,...,10:00,19:00,10:00,19:00,10:00,19:00,10:00,20:00,Closed,Closed
17733,17734,60412-294687,FR,LS,43.137051,6.004876,300 avenue de l'universite,la valette du var,U,83160,...,09:00,20:00,09:00,20:00,09:00,20:00,09:00,20:00,11:00,19:00
17734,17735,54711-288383,FR,LS,43.922646,4.912338,A7 Aire de Morieres,morieres-les-avignon,U,84310,...,07:00,20:00,07:00,20:00,07:00,20:00,07:00,20:00,07:00,20:00
17735,17736,19522-186944,FR,LS,48.897211,2.094023,19 rue de la Salle,saint germain en laye,J,78100,...,10:00,19:00,10:00,19:00,10:00,19:00,10:00,19:00,09:00,19:00


In [20]:
melody_result.to_csv('startbucks_final_Melody.csv', index=False)

In [ ]:
pd.set_option('display.max_columns', 50)